In [1]:
#Financial Computing HW2
import numpy as np
from scipy.stats import norm

#input values setting
s0 = 50
k = 50
r = 0.1
q = 0.05
sigma = 0.4
T = 0.5

In [2]:
#Black Scholes Formula
def BS(s0, k, r, q, sigma, T):
    d1 = (np.log(s0/k)+(r-q+0.5*sigma*sigma)*T)/(sigma*np.sqrt(T))
    d2 = (np.log(s0/k)+(r-q-0.5*sigma*sigma)*T)/(sigma*np.sqrt(T))
    call = s0*np.exp(-q*T)*(norm.cdf(d1))-k*np.exp(-r*T)*(norm.cdf(d2))
    put = k*np.exp(-r*T)*norm.cdf(-d2)-s0*np.exp(-q*T)*norm.cdf(-d1)
    return call, put

print('the call price is',BS(s0, k, r, q, sigma, T)[0])
print('the put price is',BS(s0, k, r, q, sigma, T)[1])

the call price is 6.039620873020631
the put price is 4.835596496639695


In [17]:
#Monte Carlo Simulation
num_simulations = 10000
num_repetitions = 20

def MC_call(s0, k, r, q, sigma, T, num_simulations, num_repetitions):
    call = []
    for i in range(num_repetitions):
        z = norm.rvs(0, 1, num_simulations)
        logst = z*sigma*np.sqrt(T) + np.log(s0)+(r-q-0.5*sigma*sigma)*T
        st = np.exp(logst)
        ct = list(map(lambda x:max(0, x-k), st))
        c0 = np.exp(-r*T)*np.mean(ct)
        call.append(c0)
        mu = np.mean(call)
        lower = mu - 2*np.std(call)
        upper = mu + 2*np.std(call)
    return mu ,[lower,upper]

def MC_put(s0, k, r, q, sigma, T, num_simulations, num_repetitions):
    put = []
    for i in range(num_repetitions):
        z = norm.rvs(0, 1, num_simulations)
        logst = z*sigma*np.sqrt(T) + np.log(s0)+(r-q-0.5*sigma*sigma)*T
        st = np.exp(logst)
        pt = list(map(lambda x:max(0, k-x), st))
        p0 = np.exp(-r*T)*np.mean(pt)
        put.append(p0)
        mu = np.mean(put)
        lower = mu - 2*np.std(put)
        upper = mu + 2*np.std(put)
    return mu ,[lower,upper]

print('the call price is', MC_call(s0, k, r, q, sigma, T, num_simulations, num_repetitions)[0])
print('the 95% confidence interval is', MC_call(s0, k, r, q, sigma, T, num_simulations, num_repetitions)[1])

print('the put price is', MC_put(s0, k, r, q, sigma, T, num_simulations, num_repetitions)[0])
print('the 95% confidence interval is', MC_put(s0, k, r, q, sigma, T, num_simulations, num_repetitions)[1])

the call price is 6.043381777054108
the 95% confidence intercal is [5.846018970271189, 6.282262326936827]
the put price is 4.847775845649094
the 95% confidence intercal is [4.735795349223837, 4.950880396728323]


In [2]:
#CRR binomial tree
#European
n = 100
dt = T/n
u = np.exp(sigma*np.sqrt(dt))
d = 1/u
p = (np.exp((r-q)*dt)-d)/(u-d)
s = np.zeros((n+1, n+1))
c = np.zeros((n+1, n+1))
put = np.zeros((n+1, n+1))
s[0, 0] = s0

for i in range(0, n+1):
    for j in range(i, n+1):
        s[i, j] = s0*(u**(j-i))*(d**i)
        c[i, n] = max(0, s[i, n]-k)
        put[i, n] = max(0, k-s[i, n])

for j in range(n-1, -1, -1):
    for i in range(0, j+1):
        c[i, j] = np.exp(-r*dt)*(p*c[i, j+1]+(1-p)*c[i+1, j+1])
        put[i, j] = np.exp(-r*dt)*(p*put[i, j+1]+(1-p)*put[i+1, j+1])
        
print('the European call price is', c[0, 0])
print('the European put price is', put[0, 0])

the European call price is 6.026046091387788
the European put price is 4.822021715006682


In [3]:
#CRR binomial tree
#American
n = 100
dt = T/n
u = np.exp(sigma*np.sqrt(dt))
d = 1/u
p = (np.exp((r-q)*dt)-d)/(u-d)
s = np.zeros((n+1, n+1))
c = np.zeros((n+1, n+1))
put = np.zeros((n+1, n+1))
s[0, 0] = s0

for i in range(0, n+1):
    for j in range(i, n+1):
        s[i, j] = s0*(u**(j-i))*(d**i)
        c[i, n] = max(0, s[i, n]-k)
        put[i, n] = max(0, k-s[i, n])

for j in range(n-1, -1, -1):
    for i in range(0, j+1):
        c[i, j] = np.exp(-r*dt)*(p*c[i, j+1]+(1-p)*c[i+1, j+1])
        c[i, j] = max(c[i, j], (s[i, j]-k))
        put[i, j] = np.exp(-r*dt)*(p*put[i, j+1]+(1-p)*put[i+1, j+1])
        put[i, j] = max(put[i, j], (k-s[i, j]))
        
print('the American call price is', c[0, 0])
print('the American put price is', put[0, 0])

the American call price is 6.026232202357999
the American put price is 4.979490435744202


In [81]:
#Bonus 1
#CRR binomial tree
#European
n = 500
dt = T/n
u = np.exp(sigma*np.sqrt(dt))
d = 1/u
p = (np.exp((r-q)*dt)-d)/(u-d)
s = np.zeros((n+1, n+1))
c = np.zeros((n+1, 1))
put = np.zeros((n+1, 1))
s[0, 0] = s0

for i in range(0, n+1):
    for j in range(i, n+1):
        s[i, j] = s0*(u**(j-i))*(d**i)
        c[i, 0] = max(0, s[i, n]-k)
        put[i, 0] = max(0, k-s[i, n])

for j in range(n-1, -1, -1):
    for i in range(0, j+1):
        c[i, 0] = np.exp(-r*dt)*(p*c[i, 0]+(1-p)*c[i+1, 0])
        put[i, 0] = np.exp(-r*dt)*(p*put[i, 0]+(1-p)*put[i+1, 0])
        

print('the European call price is', c[0, 0])
print('the European put price is', put[0, 0])

the European call price is 6.0369031580427635
the European put price is 4.83287878166121


In [82]:
#Bonus 1
#CRR binomial tree
#American
n = 500
dt = T/n
u = np.exp(sigma*np.sqrt(dt))
d = 1/u
p = (np.exp((r-q)*dt)-d)/(u-d)
s = np.zeros((n+1, n+1))
c = np.zeros((n+1, 1))
put = np.zeros((n+1, 1))
s[0, 0] = s0

for i in range(0, n+1):
    for j in range(i, n+1):
        s[i, j] = s0*(u**(j-i))*(d**i)
        c[i, 0] = max(0, s[i, n]-k)
        put[i, 0] = max(0, k-s[i, n])

for j in range(n-1, -1, -1):
    for i in range(0, j+1):
        c[i, 0] = np.exp(-r*dt)*(p*c[i, 0]+(1-p)*c[i+1, 0])
        c[i, 0] = max(c[i, 0], (s[i, j]-k))
        put[i, 0] = np.exp(-r*dt)*(p*put[i, 0]+(1-p)*put[i+1, 0])
        put[i, 0] = max(put[i, 0], (k-s[i, j]))

print('the American call price is', c[0, 0])
print('the American put price is', put[0, 0])

the American call price is 6.037102908825789
the American put price is 4.985903113057611


In [24]:
#Bonus 2
#Combination Method
n = 10000
dt = T/n
u = np.exp(sigma*np.sqrt(dt))
d = 1/u
p = (np.exp((r-q)*dt)-d)/(u-d)
call = 0
put = 0
def fac_log(n):
    result = 0
    for i in range(1, n+1):
        result += np.log(i)
    return result

for i in range(0, n+1):
    combination = np.exp(fac_log(n)-fac_log(i)-fac_log(n-i)+(n-i)*np.log(p)+i*np.log(1-p))
    call += combination*max(s0*u**(n-i)*d**i-k, 0)
    put += combination*max(k-s0*u**(n-i)*d**i, 0)

print('the European call price is', np.exp(-r*T)*call)
print('the European put price is', np.exp(-r*T)*put)

the European call price is 6.039484952998403
the European put price is 4.83546057700967
